In [ ]:
### 使用提示词完成任务

In [2]:
# 导入依赖库
import dashscope
import os

# 从环境变量中获取 API Key
dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")

print(dashscope.api_key)


# 基于 prompt 生成文本
# 使用 deepseek-v3 模型
def get_completion(prompt, model="deepseek-v3"):
    messages = [{"role": "user", "content": prompt}]  # 将 prompt 作为用户输入
    response = dashscope.Generation.call(
        model=model,
        messages=messages,
        result_format="message",  # 将输出设置为message形式
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
    )
    return response.output.choices[0].message.content  # 返回模型生成的文本


# 任务描述
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。
"""

# 用户输入
input_text = """
办个100G的套餐。
"""

# prompt 模版。instruction 和 input_text 会被替换为上面的内容
prompt = f"""
# 目标
{instruction}

# 用户输入
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# 调用大模型
response = get_completion(prompt)
print(response)

sk-9f56dae62cb445ffb2ef84e8304dcb35
==== Prompt ====

# 目标

你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。


# 用户输入

办个100G的套餐。


# 识别结果

根据用户输入"办个100G的套餐"，可以识别出以下需求：

1. **月流量需求**：明确要求100GB的流量套餐
2. **其他属性**：
   - 名称：未提及具体偏好
   - 月费价格：未提及具体金额要求

用户主要关注的是流量大小，需要提供月流量为100GB的套餐选项。价格和套餐名称方面没有特别限制。


### JSON 格式


In [3]:
# 输出格式
output_format = """
以 JSON 格式输出
"""

# 稍微调整下咒语，加入输出格式
prompt = f"""
# 目标
{instruction}

# 输出格式
{output_format}

# 用户输入
{input_text}
"""

# 调用大模型，指定用 JSON mode 输出
response = get_completion(prompt)
print(response)

```json
{
  "名称": null,
  "月费价格": null,
  "月流量": "100G"
}
```


### CoT 示例


In [ ]:
instruction = """
给定一段用户与手机流量套餐客服的对话，。
你的任务是判断客服的回答是否符合下面的规范：

- 必须有礼貌
- 必须用官方口吻，不能使用网络用语
- 介绍套餐时，必须准确提及产品名称、月费价格和月流量总量。上述信息缺失一项或多项，或信息与事实不符，都算信息不准确
- 不可以是话题终结者

已知产品包括：

经济套餐：月费50元，月流量10G
畅游套餐：月费180元，月流量100G
无限套餐：月费300元，月流量1000G
校园套餐：月费150元，月流量200G，限在校学生办理
"""

# 输出描述
output_format = """
如果符合规范，输出：Y
如果不符合规范，输出：N
"""

context = """
用户：你们有什么流量大的套餐
客服：亲，我们现在正在推广无限套餐，每月300元就可以享受1000G流量，您感兴趣吗？
"""

cot = ""
# cot = "请一步一步分析对话"

prompt = f"""
# 目标
{instruction}
{cot}

# 输出格式
{output_format}

# 对话上下文
{context}
"""

response = get_completion(prompt)
print(response)

Y


### 使用 Prompt 调优 Prompt


In [13]:
user_prompt = """
做一个手机流量套餐的客服代表，叫小瓜。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。"""

instruction = """
你是一名专业的提示词创作者。你的目标是帮助我根据需求打造更好的提示词。

你将生成以下部分：
提示词：{根据我的需求提供更好的提示词}
优化建议：{用简练段落分析如何改进提示词，需给出严格批判性建议}
问题示例：{提出最多3个问题，以用于和用户更好的交流}
"""

prompt = f"""
# 目标
{instruction}

# 用户提示词
{user_prompt}
"""

response = get_completion(prompt)
print(response)

### 提示词
"您好！我是手机流量套餐客服代表小瓜，很高兴为您服务。我们目前有以下几种流量套餐供您选择：  
1. **经济套餐**：月费50元，包含10G流量，适合日常轻度使用。  
2. **畅游套餐**：月费180元，包含100G流量，适合经常上网的用户。  
3. **无限套餐**：月费300元，包含1000G流量，适合重度流量使用者。  
4. **校园套餐**：月费150元，包含200G流量，仅限在校生办理。  
请问您目前的流量使用情况如何？或者您更倾向于哪种类型的套餐？我可以根据您的需求为您推荐最合适的方案。"

### 优化建议
1. **明确用户需求**：提示词中应更直接地询问用户的具体需求，例如每月流量使用量、预算范围或是否有特殊限制（如校园套餐）。这样可以更快地缩小选择范围，提高效率。  
2. **增加个性化推荐**：在提示词中加入一些引导性问题，例如“您是否经常观看视频或玩游戏？”或“您是否需要国际漫游服务？”这样可以更精准地推荐套餐。  
3. **简化语言**：虽然提示词已经比较清晰，但可以进一步简化语言，避免用户感到信息过载。例如，可以将套餐信息以更简洁的方式呈现，或者分步骤引导用户选择。  

### 问题示例
1. 您每月的流量使用量大概是多少？  
2. 您的预算范围是多少？  
3. 您是否有特殊需求，例如国际漫游或仅限校园使用的套餐？
